In [ ]:
from fyers_apiv3 import fyersModel

In [ ]:
client_id = 'ETYHMVN1Q6-100'
secret_key = 'I0CJUFFA03'
redirect_uri = 'https://www.google.com/'
response_type = 'code'
state = 'sample_state'

In [5]:
session = fyersModel.SessionModel(
    client_id= client_id,
    secret_key= secret_key,
    redirect_uri= redirect_uri,
    response_type= response_type
)

In [19]:
response = session.generate_authcode()

In [20]:
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=ETYHMVN1Q6-100&redirect_uri=https%3A%2F%2Fwww.google.com%2F&response_type=code&state=None


In [12]:
uri = 'https://www.google.com/?s=ok&code=200&auth_code=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiJFVFlITVZOMVE2IiwidXVpZCI6IjczYzllM2FhNWY5ZjQyMjNhMmJkZTc4MmQxNWRhOWFlIiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IllKMDg1MjYiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImF1ZCI6IltcImQ6MVwiLFwiZDoyXCIsXCJ4OjBcIixcIng6MVwiLFwieDoyXCJdIiwiZXhwIjoxNzQ0NTEwMjE1LCJpYXQiOjE3NDQ0ODAyMTUsImlzcyI6ImFwaS5sb2dpbi5meWVycy5pbiIsIm5iZiI6MTc0NDQ4MDIxNSwic3ViIjoiYXV0aF9jb2RlIn0.QHgrWlLqz-DqNLdSnyT1IR2Em421A9reNdZhnSMEdvM&state=None'

In [17]:
auth = uri.split('auth_code=')[1].split('&state')[0]

In [18]:
auth

'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiJFVFlITVZOMVE2IiwidXVpZCI6IjczYzllM2FhNWY5ZjQyMjNhMmJkZTc4MmQxNWRhOWFlIiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IllKMDg1MjYiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImF1ZCI6IltcImQ6MVwiLFwiZDoyXCIsXCJ4OjBcIixcIng6MVwiLFwieDoyXCJdIiwiZXhwIjoxNzQ0NTEwMjE1LCJpYXQiOjE3NDQ0ODAyMTUsImlzcyI6ImFwaS5sb2dpbi5meWVycy5pbiIsIm5iZiI6MTc0NDQ4MDIxNSwic3ViIjoiYXV0aF9jb2RlIn0.QHgrWlLqz-DqNLdSnyT1IR2Em421A9reNdZhnSMEdvM'